In [67]:
import pandas as pd
import numpy as np
import os, git, zipfile, shutil, sys
from inspect import currentframe, getframeinfo



def set_root_folder():
    global repo, root_folder, archive_folder, completed_folder, csv_end, normalized_end, zscore_end, list_end
    global col_num, index_start, index_end, column_index_end
    repo = git.Repo('.', search_parent_directories=True)
    root_folder = repo.working_tree_dir +"\\yeast-data\\"
    archive_folder = root_folder+"!archive\\"
    completed_folder = root_folder+"!completed\\"
    csv_end = "csv"
    normalized_end = "_normalized"
    zscore_end = "_zscore"
    list_end = "_list"
    col_num = 30 
    index_start = 0
    index_end = 16
    column_index_end = 24

def csv_file_adjust(unzipped_folder,filename):

    filetype = filename.split(".")[-1]
    if filename.endswith('csv'):
        df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")
        df.to_csv(folder+"\\"+filename.split(".")[0]+"."+csv_end)
    
def normalize_plate384(unzipped_folder,filename):
    '''Enter file to read plate and normalize data'''
    folder = unzipped_folder+"\\"
    foldername_sorted = os.path.basename(os.path.normpath(unzipped_folder))
    print(sys._getframe().f_lineno ,"foldername", foldername_sorted)
    df = pd.DataFrame([])
    df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')

    df_results_index = 0
    for column in range(col_num):
        print(sys._getframe().f_lineno ," DF column number", column, type(column), col_num)
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False)].index#].isin(['A'])].index#
            column_index = column+1
            break
        except Exception as e:
            print(sys._getframe().f_lineno, e)
            pass

    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        df_results = df_results[df_results.applymap(isnumber)]
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print(sys._getframe().f_lineno, " DF NA count:", count_na, "65 DF Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:
                print(sys._getframe().f_lineno, " DF to normalize",df_results)
                df_results = df_results.fillna(0).astype(int)
                df_results.columns = new_header
                df_median_neg_controls = int(df_results.iloc[:,0:2].stack().median())
                df_min = df_results.stack().min()
                df_max = df_results.stack().max()
                df_range = df_max-df_min
                df_results_norm = (df_results - df_min)/df_range
                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_norm
                print(sys._getframe().f_lineno, " normalized processed...",i)
            else:
                print(sys._getframe().f_lineno, " skip normalized data")
                pass
        except Exception as e: 
            print(sys._getframe().f_lineno, e)
            pass

    print(sys._getframe().f_lineno, " normalized data directory and name", unzipped_folder, "and", filename)
    new_filename = unzipped_folder+"\\"+filename#.split(".")[:-1]
    foldername_archive = unzipped_folder.split("\\")[-1]
    print(sys._getframe().f_lineno, " final file directory", new_filename,filename,foldername_sorted)
    filename_directory = completed_folder+foldername_sorted+"\\"+filename.split(".")[0]+normalized_end+"."+csv_end#filename.split(".")[-1]
    print(sys._getframe().f_lineno, " final name", filename_directory)
    df.to_csv(filename_directory)
    return(df,filename_directory)

def zscore_plate384(unzipped_folder,file_directory):
    '''Enter file to read plate and normalize data'''
    folder = unzipped_folder+"\\"
    filename = file_directory.split("\\")[-1]
    foldername_sorted = os.path.basename(os.path.normpath(unzipped_folder))
    print(sys._getframe().f_lineno, " Filename z-score",folder+filename,foldername_sorted)

    df = pd.DataFrame([])
    df = pd.read_csv(file_directory, encoding = "utf-8",sep=',')##root_folder+"!completed\\"+filename+"_normalized."+filetype,
  
    for column in range(col_num):
        print(sys._getframe().f_lineno, " z-score column number", column, type(column))
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False,regex=False)].index
            column_index = column+1
            print(sys._getframe().f_lineno, "line matching?",df_results_index)
            break
        except Exception as e:
            print(sys._getframe().f_lineno,  e)
            pass
    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()
        try:
            if  count_na < 100 and count_values > 100:

                df_results = df_results.fillna(0)
                df_results = df_results.astype(float)

                df_results.columns = new_header
                df_min = df_results.stack().min()
                df_max = df_results.stack().max()
                df_range = df_max-df_min
                df_results_norm = (df_results - df_min)/df_range

                df_mean = df_results_norm.stack().mean()
                df_std = df_results_norm.stack().std()
                df_results_zscore = (df_results_norm - df_mean)/df_std
                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_zscore
                print(sys._getframe().f_lineno, " zscore processed...",i)
            else:
                print(sys._getframe().f_lineno, " zscore skip data")
                pass
        except Exception as e: 
            print(sys._getframe().f_lineno, e)
            pass

    new_filename = unzipped_folder+"//"+filename#.split(".")[:-1]
    foldername_archive = unzipped_folder.split("\\")[-1]
    print(sys._getframe().f_lineno, " z-score final file directory", new_filename, "\nfoldername:",foldername_archive)
    filename_directory = completed_folder+foldername_sorted+"\\"+filename.split(".")[0]+zscore_end+"."+csv_end
    print(sys._getframe().f_lineno, " z-score final name", filename_directory)
    df.to_csv(filename_directory)
    return(df,filename_directory)


def index_csv(unzipped_folder,file_directory):  
    folder = unzipped_folder+"\\"
    filename = file_directory.split("\\")[-1]
    foldername_sorted = os.path.basename(os.path.normpath(unzipped_folder))
    print(sys._getframe().f_lineno, " Filename z-score",folder+filename, filetype)

    df = pd.DataFrame([])
    df = pd.read_csv(file_directory, encoding = "utf-8",sep=',')

    value_list = []
    for column in range(col_num):
        print(sys._getframe().f_lineno, " z-score column number", column, type(column))
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False,regex=False)].index
            column_index = column+1
            print(sys._getframe().f_lineno, "line matching?",df_results_index)
            break
        except Exception as e:
            print(sys._getframe().f_lineno, e)
            pass
    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()
        print(sys._getframe().f_lineno, " zscore NA count:", count_na, "Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:
                print(sys._getframe().f_lineno, " Dimensions: ",df_results.shape[0], df_results.shape[1])
                for row in range(df_results.shape[0]):
                    row_l = chr(ord('@')+row+1)
                    for col in range(df_results.shape[1]):
                        col_l = col+1
                        value_set = [str(row_l)+str(col_l), df_results.iloc[row,col]]
                        value_list.append(value_set)
        except Exception as e: 
            print(sys._getframe().f_lineno, e)
            pass
    df_list = pd.DataFrame([value_list]).transpose()
    df_list_2 = pd.DataFrame(df_list.iloc[:,0].to_list(), columns=['index','z-score'])
 
    new_filename = unzipped_folder+"\\!completed\\"+foldername_sorted#.split(".")[:-1]
    foldername_archive = unzipped_folder.split("\\")[-1]
    print(sys._getframe().f_lineno, " list final file directory", new_filename, "\nfoldername:",foldername_sorted)
    filename_directory = completed_folder+foldername_sorted+"\\"+filename.split(".")[0]+"_list."+filename.split(".")[-1]
    print(sys._getframe().f_lineno, " list final name", filename_directory)
    
    df_list_2.to_csv(filename_directory)#root_folder+"!completed\\"+filename.split(".")[0]+"_list.csv")
    return(df_list_2,filename_directory)

def unzipped_folder_access_files(plate_list, folder_archive):
    print(sys._getframe().f_lineno, "length of platelist", len(plate_list))
    n = 0
    for file in plate_list:
        filetype = file.split(".")[-1]
        first_name = file.split(".")[0]
        print(sys._getframe().f_lineno, " extracted files", first_name, filetype)
        if file.endswith("csv"):
            filename = root_folder+file
            newfilename = "".join(c for c in first_name if c not in '- ')#file.strip(" ").strip('-')
            new_filedirectory = root_folder+newfilename
            print(sys._getframe().f_lineno, filename, newfilename)
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                print(sys._getframe().f_lineno, " unzipped filename", zip_ref.filename)
                zip_ref.extractall(new_filedirectory)
                print(sys._getframe().f_lineno, zip_ref, n,new_filedirectory)
                for folder in os.listdir(new_filedirectory):
                    os.replace(new_filedirectory+"\\"+folder,new_filedirectory+"\\"+newfilename)
                    print(sys._getframe().f_lineno,   folder, new_filedirectory)
                shutil.copy(root_folder+file, folder_archive)
                zip_ref.close()
                os.remove(root_folder+file)
                n+=1
                
def new_folder_processed_data(unzipped_folder):
    foldername_sorted = os.path.basename(os.path.normpath(unzipped_folder))
    folder_unzipped_in_complete = completed_folder+foldername_sorted+"\\"
    folder_unzipped_in_archive = archive_folder+foldername_sorted+"\\"
    print(sys._getframe().f_lineno,  " folder making", folder_unzipped_in_complete)

    try:
        os.mkdir(folder_unzipped_in_complete)
        os.mkdir(folder_unzipped_in_archive)
        os.chdir(root_folder)

    except:
        os.chdir(root_folder)

set_root_folder()
plate_list = os.listdir(root_folder)
unzipped_folder_access_files(plate_list, archive_folder)
unzipped_folders_list = os.listdir(root_folder)
print(sys._getframe().f_lineno,   unzipped_folders_list)

for unzipped_foldername in unzipped_folders_list:
    if not any(extension in unzipped_foldername for extension in ("!", "zip")):
        foldername_extracted = unzipped_foldername.split("-")[0]
        print(sys._getframe().f_lineno,  " unzipped folder",unzipped_foldername,foldername_extracted)
        unzipped_folder_directory = root_folder+unzipped_foldername+"\\"+unzipped_foldername+"\\"
        print(sys._getframe().f_lineno,  " unzipped folder final", unzipped_folder_directory)
        new_folder_processed_data(unzipped_folder_directory)
        unzipped_folder = os.listdir(unzipped_folder_directory)
        print(sys._getframe().f_lineno, " unzipped folder list", unzipped_folder)

        for unzipped_file in unzipped_folder:
            if unzipped_file.endswith('csv'):
                print(sys._getframe().f_lineno,  " unzipped files", unzipped_file)
                normalized_directory = normalize_plate384(unzipped_folder_directory,unzipped_file)
                print(sys._getframe().f_lineno,  " normalized directory", normalized_directory[1])
                z_score_directory = zscore_plate384(unzipped_folder_directory,normalized_directory[1])
                index_csv(unzipped_folder_directory,z_score_directory[1])
            else:
                print(sys._getframe().f_lineno, " end", unzipped_foldername, unzipped_file)
                pass

257  list ['!archive', '!completed', 'PIGWplate1&19repeat090720230907T203221Z001', 'PIGWRepeat83020230830T215756Z001', 'PIGWRepeat83020230831T194039Z001']
215 length of platelist 5
220  extracted files !archive !archive
220  extracted files !completed !completed
220  extracted files PIGWplate1&19repeat090720230907T203221Z001 PIGWplate1&19repeat090720230907T203221Z001
220  extracted files PIGWRepeat83020230830T215756Z001 PIGWRepeat83020230830T215756Z001
220  extracted files PIGWRepeat83020230831T194039Z001 PIGWRepeat83020230831T194039Z001
262 ['!archive', '!completed', 'PIGWplate1&19repeat090720230907T203221Z001', 'PIGWRepeat83020230830T215756Z001', 'PIGWRepeat83020230831T194039Z001']
266  unzipped folder PIGWplate1&19repeat090720230907T203221Z001 PIGWplate1&19repeat090720230907T203221Z001
268  unzipped folder final C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWplate1&19repeat090720230907T203221Z001\PIGWplate1&19repeat090720230907T203221Z001\
244  folder making C:\Users\vandu\Desktop\

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because t

171  z-score column number 0 <class 'int'>
178 Can only use .str accessor with string values!
171  z-score column number 1 <class 'int'>
175 line matching? Int64Index([22, 23, 24, 25, 58, 77, 87, 101, 105, 117, 118], dtype='int64')
186  zscore NA count: 331 Values count: 53 done
186  zscore NA count: 329 Values count: 55 done
186  zscore NA count: 333 Values count: 51 done
186  zscore NA count: 334 Values count: 50 done
186  zscore NA count: 108 Values count: 276 done
186  zscore NA count: 324 Values count: 60 done
186  zscore NA count: 333 Values count: 51 done
186  zscore NA count: 338 Values count: 46 done
186  zscore NA count: 309 Values count: 51 done
186  zscore NA count: 51 Values count: 21 done
186  zscore NA count: 28 Values count: 20 done
204  list final file directory C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWRepeat83020230830T215756Z001\PIGWRepeat83020230830T215756Z001\\!completed\PIGWRepeat83020230830T215756Z001 
foldername: PIGWRepeat83020230830T215756Z001
206  list

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because t

140  zscore skip data
140  zscore skip data
140  zscore skip data
140  zscore skip data
140  zscore skip data
140  zscore skip data
140  zscore skip data
148  z-score final file directory C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWRepeat83020230830T215756Z001\PIGWRepeat83020230830T215756Z001\//13_normalized.csv 
foldername: 
150  z-score final name C:\Users\vandu\Desktop\vancanwin\yeast-data\!completed\PIGWRepeat83020230830T215756Z001\13_normalized_zscore.csv
161  Filename z-score C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWRepeat83020230830T215756Z001\PIGWRepeat83020230830T215756Z001\\13_normalized_zscore.csv csv
171  z-score column number 0 <class 'int'>
178 Can only use .str accessor with string values!
171  z-score column number 1 <class 'int'>
175 line matching? Int64Index([22, 23, 24, 25, 58, 77, 87, 101, 105, 117, 118], dtype='int64')
186  zscore NA count: 331 Values count: 53 done
186  zscore NA count: 329 Values count: 55 done
186  zscore NA count: 333 Values count: 5

...

161  Filename z-score C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWRepeat83020230831T194039Z001\PIGWRepeat83020230831T194039Z001\\4_normalized_zscore.csv csv
171  z-score column number 0 <class 'int'>
178 Can only use .str accessor with string values!
171  z-score column number 1 <class 'int'>
175 line matching? Int64Index([22, 23, 24, 25, 58, 77, 87, 101, 105, 117, 118], dtype='int64')
186  zscore NA count: 331 Values count: 53 done
186  zscore NA count: 329 Values count: 55 done
186  zscore NA count: 333 Values count: 51 done
186  zscore NA count: 334 Values count: 50 done
186  zscore NA count: 108 Values count: 276 done
186  zscore NA count: 324 Values count: 60 done
186  zscore NA count: 333 Values count: 51 done
186  zscore NA count: 338 Values count: 46 done
186  zscore NA count: 309 Values count: 51 done
186  zscore NA count: 51 Values count: 21 done
186  zscore NA count: 28 Values count: 20 done
204  list final file directory C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWR

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')


186  zscore NA count: 331 Values count: 53 done
186  zscore NA count: 329 Values count: 55 done
186  zscore NA count: 333 Values count: 51 done
186  zscore NA count: 334 Values count: 50 done
186  zscore NA count: 108 Values count: 276 done
186  zscore NA count: 324 Values count: 60 done
186  zscore NA count: 333 Values count: 51 done
186  zscore NA count: 338 Values count: 46 done
186  zscore NA count: 309 Values count: 51 done
186  zscore NA count: 51 Values count: 21 done
186  zscore NA count: 28 Values count: 20 done
204  list final file directory C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWRepeat83020230831T194039Z001\PIGWRepeat83020230831T194039Z001\\!completed\PIGWRepeat83020230831T194039Z001 
foldername: PIGWRepeat83020230831T194039Z001
206  list final name C:\Users\vandu\Desktop\vancanwin\yeast-data\!completed\PIGWRepeat83020230831T194039Z001\6_normalized_zscore_list.csv
275  unzipped files 7.csv
36 foldername PIGWRepeat83020230831T194039Z001
43  DF column number 0 <class 

C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep="[_,\t ;|:]")#,sep ="[_\t ;|:]",engine='python')
C:\Users\vandu\AppData\Local\Temp\ipykernel_10920\3619917226.py:39: ParserWarning: Falling back to the 'python' engine because t

161  Filename z-score C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWRepeat83020230831T194039Z001\PIGWRepeat83020230831T194039Z001\\9_normalized_zscore.csv csv
171  z-score column number 0 <class 'int'>
178 Can only use .str accessor with string values!
171  z-score column number 1 <class 'int'>
175 line matching? Int64Index([22, 23, 24, 25, 58, 77, 87, 101, 105, 117, 118], dtype='int64')
186  zscore NA count: 331 Values count: 53 done
186  zscore NA count: 329 Values count: 55 done
186  zscore NA count: 333 Values count: 51 done
186  zscore NA count: 334 Values count: 50 done
186  zscore NA count: 108 Values count: 276 done
186  zscore NA count: 324 Values count: 60 done
186  zscore NA count: 333 Values count: 51 done
186  zscore NA count: 338 Values count: 46 done
186  zscore NA count: 309 Values count: 51 done
186  zscore NA count: 51 Values count: 21 done
186  zscore NA count: 28 Values count: 20 done
204  list final file directory C:\Users\vandu\Desktop\vancanwin\yeast-data\PIGWR